0. Paste token in form below.
1. Install. 
2. If you see an error concerning tokenizers or sentencepiece not being installed properly, simply press **runtime** and **restart and run all** and it will work.



In [ ]:
token = "" # @param {"type": "string"}

In [ ]:
# !pip install -q relevanceai==2.5.4
# !pip install -q langdetect
# !pip install -q sentencepiece
# !pip install -q transformers

from relevanceai.utils import decode_workflow_token

config = decode_workflow_token(token)
print(config)
fields = config['fields']
dataset_id = config['dataset_id']

from relevanceai import Client
try:
  import os
  os.environ['WORKFLOW_ID'] = job_id
except Exception as e:
  print(e)

client = Client(config['authorizationToken'])
ds = client.Dataset(dataset_id)
ds.translate(fields=fields)